In [21]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle


In [22]:
model=load_model('churn_model.h5')

with open('le_gender.pkl', 'rb') as file:
    le_gender = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('ohe_geography.pkl', 'rb') as file:
    ohe_geography = pickle.load(file)

In [23]:
input_data = {
    'CreditScore': 700,
    'Geography': 'Germany',
    'Gender': 'Female',
    'Age': 30,
    'Tenure': 5,
    'Balance': 70000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 60000
}

In [24]:
geo_value = input_data.get('Geography')
geo_array = np.array([[geo_value]])
try:
    geo_encoded = ohe_geography.transform(geo_array)
    if hasattr(geo_encoded, 'toarray'):
        geo_encoded = geo_encoded.toarray()
except Exception as e:

    categories = getattr(ohe_geography, 'categories_', None)
    if categories is not None and len(categories) > 0:
        cols = list(categories[0])
        geo_encoded = np.zeros((1, len(cols)), dtype=int)
        if geo_value in cols:
            geo_encoded[0, cols.index(geo_value)] = 1
    else:
       
        tmp = pd.get_dummies([geo_value], prefix='Geography')
        if isinstance(tmp, pd.DataFrame):
            geo_encoded = tmp.values
        else:
            geo_encoded = np.array(tmp)

if hasattr(ohe_geography, 'get_feature_names_out'):
    try:
        cols = ohe_geography.get_feature_names_out(['Geography'])
    except Exception:
        cols = [f'Geography_{c}' for c in getattr(ohe_geography, 'categories_', [[]])[0]]
else:
    cols = [f'Geography_{i}' for i in range(geo_encoded.shape[1])]
geo_encoded_df = pd.DataFrame(geo_encoded, columns=cols)
geo_encoded_df

d:\ChurnOff\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [25]:
input_df=pd.DataFrame([input_data])

In [26]:
input_df['Gender'] = le_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,700,Germany,0,30,5,70000,2,1,1,60000


In [27]:
input_df=pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)

In [28]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,0,30,5,70000,2,1,1,60000,0.0,1.0,0.0


In [30]:
input_scale=scaler.transform(input_df)
input_scale

array([[ 5.01787943e-01, -1.09499335e+00, -8.45930770e-01,
        -1.34471949e-03, -9.77012936e-02,  8.08436154e-01,
         6.49202671e-01,  9.74816989e-01, -7.02965505e-01,
        -9.98501123e-01,  1.72572313e+00, -5.76388018e-01]])

In [31]:
predict=model.predict(input_scale)
predict

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


array([[0.02585464]], dtype=float32)

In [32]:
predict_prob=predict[0][0]
predict_prob

np.float32(0.025854642)

In [33]:
if predict_prob > 0.5:
    print(f'The customer is likely to churn with a probability of {predict_prob:.2f}')
else:
    print(f'The customer is unlikely to churn with a probability of {predict_prob:.2f}')


The customer is unlikely to churn with a probability of 0.03
